## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions
0,0,Saldanha,ZA,-33.01,17.94,48.2,81,98,1.12,overcast clouds
1,1,Qeshm,IR,26.96,56.27,89.6,74,20,11.41,few clouds
2,2,Pangnirtung,CA,66.15,-65.71,44.6,93,90,2.95,light intensity drizzle
3,3,Kruisfontein,ZA,-34.00,24.73,59.0,87,100,3.00,overcast clouds
4,4,Tambacounda,SN,13.75,-13.76,82.4,83,100,6.93,overcast clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions
1,1,Qeshm,IR,26.96,56.27,89.60,74,20,11.41,few clouds
4,4,Tambacounda,SN,13.75,-13.76,82.40,83,100,6.93,overcast clouds
5,5,Dakoro,NE,14.51,6.76,79.38,73,57,16.40,heavy intensity rain
9,9,Jamestown,US,42.10,-79.24,78.01,65,1,6.93,clear sky
10,10,Bowmore,US,34.94,-79.30,77.00,88,75,5.82,light rain
11,11,Robertson,US,36.52,-86.87,87.80,52,1,2.44,clear sky
24,24,Alvorada,BR,-12.48,-49.12,84.40,36,81,4.23,broken clouds
31,31,Luziania,BR,-16.25,-47.95,75.20,50,82,9.17,broken clouds
32,32,Melilla,ES,35.29,-2.94,78.80,83,26,4.70,scattered clouds
33,33,Bonthe,SL,7.53,-12.51,76.60,85,98,12.15,overcast clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID               0
City                  0
Country               0
Lat                   0
Lng                   0
Max Temp              0
Humidity              0
Cloudiness            0
Wind Speed            0
Current Conditions    0
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Conditions
1,1,Qeshm,IR,26.96,56.27,89.60,74,20,11.41,few clouds
4,4,Tambacounda,SN,13.75,-13.76,82.40,83,100,6.93,overcast clouds
5,5,Dakoro,NE,14.51,6.76,79.38,73,57,16.40,heavy intensity rain
9,9,Jamestown,US,42.10,-79.24,78.01,65,1,6.93,clear sky
10,10,Bowmore,US,34.94,-79.30,77.00,88,75,5.82,light rain
...,...,...,...,...,...,...,...,...,...,...
546,546,Maraa,BR,-1.83,-65.37,77.43,59,20,1.81,few clouds
547,547,Parana,AR,-31.73,-60.52,75.00,48,7,1.99,clear sky
548,548,Nioro,GM,13.35,-15.75,76.66,93,100,6.24,light rain
550,550,Suluktu,KG,39.93,69.57,75.20,44,46,20.13,scattered clouds


In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Qeshm,IR,89.60,26.96,56.27,
4,Tambacounda,SN,82.40,13.75,-13.76,
5,Dakoro,NE,79.38,14.51,6.76,
9,Jamestown,US,78.01,42.10,-79.24,
10,Bowmore,US,77.00,34.94,-79.30,
11,Robertson,US,87.80,36.52,-86.87,
24,Alvorada,BR,84.40,-12.48,-49.12,
31,Luziania,BR,75.20,-16.25,-47.95,
32,Melilla,ES,78.80,35.29,-2.94,
33,Bonthe,SL,76.60,7.53,-12.51,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
hotel_df.head(120)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Qeshm,IR,89.60,26.96,56.27,
4,Tambacounda,SN,82.40,13.75,-13.76,
5,Dakoro,NE,79.38,14.51,6.76,
9,Jamestown,US,78.01,42.10,-79.24,
10,Bowmore,US,77.00,34.94,-79.30,
...,...,...,...,...,...,...
382,Najran,SA,86.00,17.49,44.13,
384,Sitia,GR,75.99,35.20,26.10,
389,Marathon,US,78.80,44.90,-89.77,
390,Bridgetown,BB,82.40,13.10,-59.62,


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Qeshm,IR,89.60,26.96,56.27,
4,Tambacounda,SN,82.40,13.75,-13.76,
5,Dakoro,NE,79.38,14.51,6.76,
9,Jamestown,US,78.01,42.10,-79.24,
10,Bowmore,US,77.00,34.94,-79.30,
...,...,...,...,...,...,...
546,Maraa,BR,77.43,-1.83,-65.37,
547,Parana,AR,75.00,-31.73,-60.52,
548,Nioro,GM,76.66,13.35,-15.75,
550,Suluktu,KG,75.20,39.93,69.57,


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [24]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))